In [2]:
import requests
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import csv
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from nltk import pos_tag
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [3]:
##SCRAPER CODE
# names = []
# reviews = []
# grades = []
# url = 'https://www.temptalia.com/category/reviews/page/1035/'
# response = requests.get(url)
# page = response.text
# soup = BeautifulSoup(page, "lxml")
# product_links = []
# links = soup.find_all(class_='btn btn-secondary btn-lg more-link')
# for link in links:
#     product_links.append(link.get('href'))
# for link in product_links:
#     response = requests.get(link)
#     soup = BeautifulSoup(response.text, "lxml")
#     product_names = soup.find_all(class_ = 'hidden-md-up text-xs-center text-md-left text-uppercase pt-1 mb-2')
#     product_reviews = (soup.find_all('p'))
#     product_grades = (soup.find_all(class_ = 'glossover-grade f-7 bold'))
#     counter = 0
#     for review in product_reviews:
#         if '$' in review.text and len(review.text) > 35:
#             reviews.append(review.text)
#             names.append(product_names[counter].text)
#             grades.append(product_grades[counter].text)
#             counter += 1
# new_df = pd.DataFrame(data = {'names': names, 'reviews': reviews, 'grades': grades})
# new_df.to_csv('temp_sheet.csv')
# new_df = new_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# new_df.tail()

In [4]:
new_df = pd.read_csv('output.csv', encoding = "ISO-8859-1")

In [5]:
new_df.tail()

,Brand,Name,Product,Review,Grade
5168,MAC,OVERSPICED,Lipgloss,"a dirty muted coral. ItÕs a warm-toned, light-...",C+
5169,MAYBELLINE,NUDE ILLUSION (060),Lipgloss,"light-medium peachy beige with soft, warm unde...",B
5170,SURRATT BEAUTY,AU NATURELLE,Lipgloss,a universal neutral with a kiss of gold. ItÕs ...,C+
5171,CHANEL,SENSUEL (11),Lipgloss,a peachy brown with a very fine gold shimmer. ...,B
5172,NARS,BELLE DE JOUR,Lipstick,"a light-medium, yellowed peach with warm under...",D


In [6]:
def removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

In [7]:
clean = new_df['Review'].apply(removeNonAscii)

In [8]:
new_df['Review'] = clean

In [9]:
new_df.replace('-', ' ', regex=True, inplace=True)

In [10]:
new_df.sample(n=20)

,Brand,Name,Product,Review,Grade
4363,DIOR,HOUSE OF PINKS,Palette,a pink hued palette with five shades ranging f...,D+
138,COLOUR POP,ROUND MIDIGHT,Lipgloss,"a pale, milky pink base with barely there pink...",B
4535,BOBBI BROWN,LADY RUBA,Lipstick,"a bright, reddish coral. Its a bold, orange re...",A
2124,MAKE UP FOR EVER,1 FROZEN SILVER,Highlighter,"a light, bright silver with cooler undertones ...",A
4222,CHANEL,LEGENDE (428),Lipstick,"an intense satin pink. Its a medium, muted pin...",B+
3126,BURBERRY,BRIGHT CRIMSON,Lipstick,"a bright, medium pink coral with warm underton...",A
4382,MAYBELLINE,LUST FOR MAUVE (535),Lipgloss,a rosy plum with warm undertones and beige and...,B
2173,BITE BEAUTY,FLAMBÃ©,Lipstick,"a vibrant, medium orange with warm, red undert...",A
1907,TOM FORD BEAUTY,ZELDA,Lipstick,"a deep fuchsia with cool, blue undertones and ...",B+
4161,LORAC,BEIGE HIGHLIGHT,Palette,three highlighters and three contouring powder...,A


In [11]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', 
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)
cv_data = count_vectorizer.fit_transform(new_df['Review'])
lsa_ = TruncatedSVD(n_components=357)
lsa_data = lsa_.fit_transform(cv_data)

In [15]:
new_vec = lsa_.transform(count_vectorizer.transform(['a new, limited edition neutral-hued palette that leans slightly more mauve/plum than overtly warm-toned. Of the three palettes launched in the Art Library collection, this has the most repeated shades (past releases and/or permanent shades). If you feel like a lot of matte eyeshadows are too powdery, you may you enjoy MAC’s formula, which tends to be firmer, a little stiffer and better for gradually building color with little to no fallout. Most of the mattes were blendable, but the darker shades–Sugar Plum and Dance in the Dark–could have been easier to soften along the edges. LES Artiste had some fallout and a tendency to emphasize my lid texture, so I’d recommend using fingertips or a wet brush to apply it, and I felt like Nude Model was overly slippery.']))
nn = NearestNeighbors(n_neighbors=1, metric='cosine')
nn.fit(lsa_data)
results = nn.kneighbors(new_vec, n_neighbors=5)


In [13]:
for i in range(len(results[1][0])):
        print(new_df.iloc[results[1][0][i]])

Brand                                                    MAC
Name                                  THE BEST OF EVERYTHING
Product                                              Palette
Review     a new, limited edition smoky palette. I love t...
Grade                                                     B 
Name: 3112, dtype: object
Brand                                                    MAC
Name                                            OH MY BANANA
Product                                              Palette
Review     a brighter mix of warm toned shades from pale ...
Grade                                                     B+
Name: 2555, dtype: object
Brand                                              TOO FACED
Name                                            NATURAL LOVE
Product                                              Palette
Review     30 eyeshadows in varying shades of pinks, gold...
Grade                                                     B 
Name: 2715, dtype: object
Brand  

In [14]:
def get_nearest_prods():
    new_vec = lsa_.transform(count_vectorizer.transform([input()]))
    results = nn.kneighbors(new_vec, n_neighbors=5)
    rec_strings = []
    for i in range(len(results[1][0])):
        rec_strings.append(f"Try {new_df.iloc[results[1][0][i]]['Name']}, a {new_df.iloc[results[1][0][i]]['Product']} by {new_df.iloc[results[1][0][i]]['Brand']}.")
    return rec_strings

    

In [15]:
get_nearest_prods()

So i sweat very easily, I'm going through a lot of blood work and seeing an endocrinologist soon to figure out everything. The thing is this makes it very hard to wear blush. When I sweat the blush turns streaky. I'm going to a concert Sunday evening and I really want to wear blush but I'm afraid that it will turn streaky if I sweat. I'll be standing in the general admission pit area on the floor. The only blushes I have right now are MAC blushes and a Lancoke blush. All I usually wear is foundation, powder, and lip gloss. Unfortunately I can't wear eye makeup at all due to an eye condition. The only thing I have tried is the Note Wild Photofocus Natural Finish Spray. Does anyone have any tips on how to keep my blush from streaking? Thanks in advance. ,😊


['Try PIN UP GIRL, a Palette by TARTE.',
 'Try COPPER FACE COMPACT, a Palette by MAC.',
 'Try TECHNO CUMBIA, a Palette by MAC.',
 'Try RADIANT, a Palette by ANASTASIA.',
 'Try ALPEN GLOW (06), a Blush by SHISEIDO.']

In [12]:
import pickle
pickle.dump(count_vectorizer, open('vectorizer.pkl', 'wb'))

In [13]:
pickle.dump(lsa_, open('dim_red.pkl', 'wb'))

In [16]:
pickle.dump(nn, open('model.pkl', 'wb'))

In [17]:
pickle.dump(new_df, open('reviews.pkl', 'wb'))